In [4]:
import cv2
import torch
import time
import os
import numpy as np

from utils.inference.image_processing import crop_face, get_final_image, show_images, get_only_swaped_image
from utils.inference.video_processing import read_video, get_target, get_final_video, add_audio_from_another_video, face_enhancement
from utils.inference.core import model_inference

from network.AEI_Net import AEI_Net
from coordinate_reg.image_infer import Handler
from insightface_func.face_detect_crop_multi import Face_detect_crop
from arcface_model.iresnet import iresnet100
from models.pix2pix_model import Pix2PixModel
from models.config_sr import TestOptions

/home/ubuntu-gpu/project/laboratory/ghost-train/.venv/lib/python3.8/site-packages/kornia/augmentation/augmentation.py:1830: DeprecationWarning: GaussianBlur is no longer maintained and will be removed from the future versions. Please use RandomGaussianBlur instead.
  warnings.warn(
/home/ubuntu-gpu/project/laboratory/ghost-train/.venv/lib/python3.8/site-packages/mxnet/numpy/utils.py:37: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bool = onp.bool
/home/ubuntu-gpu/project/laboratory/ghost-train/.venv/lib/python3.8/site-packages/mxnet/numpy/fallback.py:143: DeprecationWarning: In accordance with NEP 32, the function mirr was removed from NumPy version 1.20.  A replace

In [5]:
from typing import List
import numpy as np
import matplotlib.pyplot as plt
import base64
from io import BytesIO
from typing import Callable, List

import numpy as np
import torch
import cv2
from utils.inference.masks import face_mask_static 
from matplotlib import pyplot as plt
from insightface.utils import face_align

def show_face_frames(final_frames_list: List[List[np.ndarray]], 
                      titles: List[str] = None, 
                      figsize: tuple = (20, 5), 
                      fontsize: int = 15):
    """
    オプションのタイトルと共に最終フレームを表示する

    Args:
    final_frames_list (List[List[np.ndarray]]): 最終フレームのリストのリスト
    titles (List[str], optional): 各セットのタイトル。デフォルトはNone
    figsize (tuple, optional): 図のサイズ。デフォルトは(20, 5)
    fontsize (int, optional): タイトルのフォントサイズ。デフォルトは15
    """

    fig, axes = plt.subplots(1, len(final_frames_list), figsize=figsize)
    for idx, (ax, image) in enumerate(zip(axes, final_frames_list)):
        ax.imshow(image[0][:, :, ::-1])
        if titles:
            ax.set_title(titles[idx], fontsize=fontsize)
        ax.axis("off")
    plt.show()

def get_only_swaped_fullimage(final_frames: List[np.ndarray],
                          crop_frames: List[np.ndarray],
                          full_frame: np.ndarray,
                          tfm_arrays: List[np.ndarray]):
    """
    フェイススワッピングや他の画像変換の結果を用いて最終的な画像を生成する

    Args:
    final_frames (List[np.ndarray]): 最終的に生成されたフレームのリスト
    full_frame (np.ndarray): フルサイズの元のフレーム
    tfm_arrays (List[np.ndarray]): 各フレームに適用された変換行列のリスト
    """
    final = full_frame.copy()
    
    for frames, crop_frames, tfm_arrays in zip(final_frames_list, crop_frames_list, tfm_array_list):
        for frame, crop_frame, tfm_array in zip(frames, crop_frames, tfm_arrays):
            if frame.size > 0:
                frame_resized = cv2.resize(frame, (crop_frame.shape[1], crop_frame.shape[0]))
                mat_rev = cv2.invertAffineTransform(tfm_array)
                swap_t = cv2.warpAffine(frame_resized, mat_rev, (full_frame.shape[1], full_frame.shape[0]), borderMode=cv2.BORDER_REPLICATE)
                mask = cv2.warpAffine(np.ones_like(crop_frame[:, :, 0], dtype=np.uint8), mat_rev, (full_frame.shape[1], full_frame.shape[0]))
                mask = mask[:, :, np.newaxis]
                final = mask * swap_t + (1 - mask) * final
    
    final = np.array(final, dtype='uint8')
    return final

def get_only_swaped_faceimage(final_frames: List[np.ndarray],
                          crop_frames: List[np.ndarray],
                          full_frame: np.ndarray,
                          tfm_arrays: List[np.ndarray]):
    """
    フェイススワッピングや他の画像変換の結果を用いて最終的な画像を生成する

    Args:
    final_frames (List[np.ndarray]): 最終的に生成されたフレームのリスト
    full_frame (np.ndarray): フルサイズの元のフレーム
    tfm_arrays (List[np.ndarray]): 各フレームに適用された変換行列のリスト
    """
    # 各フレームに対応するパラメータのリスト（初期値はすべてNone）
    params = [None for i in range(len(final_frames))]
    
    # 各フレームについてループ
    for i in range(len(final_frames)):
        # 224x224ピクセルにリサイズされたフレームを取得
        frame = cv2.resize(final_frames[i][0], (224, 224))
        
        # リサイズされたフレームと対応するクロップフレームからランドマークを検出
        landmarks = handler.get_without_detection_without_transform(frame)     
        landmarks_tgt = handler.get_without_detection_without_transform(crop_frames[i][0])

        # face_mask_static関数を使用して、マスクと他の出力を取得
        mask, _ = face_mask_static(crop_frames[i][0], landmarks, landmarks_tgt, params[i])
        # 変換行列を反転
        mat_rev = cv2.invertAffineTransform(tfm_arrays[i][0])

        # リサイズされたフレームとマスクを元のフルサイズのフレームに再変換
        #swap_t = cv2.warpAffine(frame, mat_rev, (full_frame.shape[1], full_frame.shape[0]), borderMode=cv2.BORDER_REPLICATE)
        #mask_t = cv2.warpAffine(mask, mat_rev, (full_frame.shape[1], full_frame.shape[0]))
        #mask_t = np.expand_dims(mask_t, 2)

        # マスクを使用して変換されたフレームを最終フレームに適用
        final = mask*frame + (1-mask)*crop_frames[i][0]
    
    # 最終フレームをuint8型の配列に変換し、関数の結果として返す
    final = np.array(final, dtype='uint8')
    return final

import matplotlib.pyplot as plt
import numpy as np
from typing import List

def noshow_images(images: List[np.ndarray], 
                titles=None, 
                figsize=(20, 5), 
                fontsize=15):
    """
    Display images with optional titles
    オプションのタイトルと共に画像を表示する
    """
    if titles:
        assert len(titles) == len(images), "Amount of images should be the same as the amount of titles"
    
    fig, axes = plt.subplots(1, len(images), figsize=figsize)
    plt.subplots_adjust(wspace=0, hspace=0)  # 余白をなくす設定

    for idx, (ax, image) in enumerate(zip(axes, images)):
        ax.imshow(image[:, :, ::-1])
        if titles:
            ax.set_title(titles[idx], fontsize=fontsize)
        ax.axis("off")

    plt.tight_layout(pad=0)  # 余白をなくす設定
    plt.show()

In [11]:
def swap(G_path, target_path, source_path, save_path):
    app = Face_detect_crop(name='antelope', root='./insightface_func/models')
    app.prepare(ctx_id= 0, det_thresh=0.6, det_size=(640,640))

    # main model for generation
    G = AEI_Net(backbone='unet', num_blocks=2, c_id=512)
    G.eval()
    G.load_state_dict(torch.load(G_path, map_location=torch.device('cpu')))
    G = G.cuda()
    G = G.half()

    # arcface model to get face embedding
    netArc = iresnet100(fp16=False)
    netArc.load_state_dict(torch.load('arcface_model/backbone.pth'))
    netArc = netArc.cuda()
    netArc.eval()

    # model to get face landmarks
    handler = Handler('./coordinate_reg/model/2d106det', 0, ctx_id=0, det_size=640)

    use_sr = False
    if use_sr:
        os.environ['CUDA_VISIBLE_DEVICES'] = '0'
        torch.backends.cudnn.benchmark = True
        opt = TestOptions()
        model = Pix2PixModel(opt)
        model.netG.train()

    image_to_image = True

    if image_to_image:
        path_to_target = target_path
    else:
        path_to_video = 'examples/videos/random_gif.gif'

    source_full = cv2.imread(source_path)
    OUT_VIDEO_NAME = "examples/results/result.mp4"
    crop_size = 224
    BS = 60

    try:
        source = crop_face(source_full, app, crop_size)[0]
        source = [source[:, :, ::-1]]
        print("Everything is ok!")
    except TypeError:
        print("Bad source images")

    if image_to_image:
        target_full = cv2.imread(path_to_target)
        full_frames = [target_full]
        print(len(full_frames))
    else:
        full_frames, fps = read_video(path_to_video)

    target = get_target(full_frames, app, crop_size)

    START_TIME = time.time()
    final_frames_list, crop_frames_list, full_frames, tfm_array_list = model_inference(full_frames,
                                                                                      source,
                                                                                      target,
                                                                                      netArc,
                                                                                      G,
                                                                                      app,
                                                                                      set_target=False,
                                                                                      crop_size=crop_size,
                                                                                      BS=BS)
    
    result = get_final_image(final_frames_list, crop_frames_list, full_frames[0], tfm_array_list, handler)
    
    # 生成した画像を保存
    #save_result_path = f"{save_path}/result.png"
    #cv2.imwrite(save_result_path, result)
    

    return result, source, target_full




In [14]:
import os

def get_image_list(folder_path):
    # 指定フォルダ内のすべての画像ファイルのパスを取得
    return [os.path.join(folder_path, f) for f in os.listdir(folder_path) if f.endswith(('.png', '.jpg', '.jpeg'))]

# フォルダのパスを指定
target_folder = 'examples/quantitative-evaluation/target_images/'
source_folder = 'examples/quantitative-evaluation/source_images/'

# フォルダ内のすべての画像をリストに取得
target_list = get_image_list(target_folder)
source_list = get_image_list(source_folder)

save_path = "examples/results/id85-enhanced9"

# G_path を指定
G_path = 'saved_weights/current_models/stage3_identity_enhancesd_9/G_2_000000.pth'

# source 画像を固定し、すべての target 画像との組み合わせを実行
for j, source_path in enumerate(source_list):
    for i, target_path in enumerate(target_list):
        # G_path のみを使用して swap 処理を行う
        result, source, target_full = swap(G_path, target_path, source_path, save_path)

        # 生成結果の保存
        result_image_path = f'{save_path}/result_source{j}_target{i}.png'
        cv2.imwrite(result_image_path, result)
        print(f"Image saved at {result_image_path}")
        # 結果の画像表示
        #noshow_images([source[0][:, :, ::-1], target_full, result], ['Source Image', 'Target Image', 'GHOST'], figsize=(20, 15))


input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
loading ./coordinate_reg/model/2d106det 0


[02:14:46] /work/mxnet/src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.5.0. Attempting to upgrade...
[02:14:46] /work/mxnet/src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!


input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
Everything is ok!
1


100%|██████████| 1/1 [00:00<00:00, 18.78it/s]
1it [00:00, 768.47it/s]
1it [00:00, 5592.41it/s]
100%|██████████| 1/1 [00:00<00:00, 10180.35it/s]
/home/ubuntu-gpu/project/laboratory/ghost-train/.venv/lib/python3.8/site-packages/mxnet/ndarray/utils.py:141: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  if spsp is not None and isinstance(source_array, spsp.csr.csr_matrix):
/home/ubuntu-gpu/project/laboratory/ghost-train/.venv/lib/python3.8/site-packages/mxnet/ndarray/utils.py:141: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  if spsp is not None and isinstance(source_array, spsp.csr.csr_matrix):


Image saved at examples/results/id85-enhanced9/result_source0_target0.png
input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
loading ./coordinate_reg/model/2d106det 0


...

[02:25:28] /work/mxnet/src/nnvm/legacy_json_util.cc:208: Loading symbol saved by previous version v1.5.0. Attempting to upgrade...
[02:25:28] /work/mxnet/src/nnvm/legacy_json_util.cc:216: Symbol successfully upgraded!


input mean and std: 127.5 127.5
find model: ./insightface_func/models/antelope/glintr100.onnx recognition
find model: ./insightface_func/models/antelope/scrfd_10g_bnkps.onnx detection
set det-size: (640, 640)
Everything is ok!
1


100%|██████████| 1/1 [00:00<00:00, 17.92it/s]
1it [00:00, 173.48it/s]
1it [00:00, 6297.75it/s]
100%|██████████| 1/1 [00:00<00:00, 22919.69it/s]


Image saved at examples/results/id85-enhanced9/result_source11_target16.png


/home/ubuntu-gpu/project/laboratory/ghost-train/.venv/lib/python3.8/site-packages/mxnet/ndarray/utils.py:141: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  if spsp is not None and isinstance(source_array, spsp.csr.csr_matrix):
/home/ubuntu-gpu/project/laboratory/ghost-train/.venv/lib/python3.8/site-packages/mxnet/ndarray/utils.py:141: DeprecationWarning: Please use `csr_matrix` from the `scipy.sparse` namespace, the `scipy.sparse.csr` namespace is deprecated.
  if spsp is not None and isinstance(source_array, spsp.csr.csr_matrix):


In [ ]:
# １．顔のクローズアップ
face_close_up = final_frames_list.copy()

# 後処理
#if True:
#    final_frames_list = get_only_swaped_faceimage(final_frames_list, crop_frames_list, full_frames[0], tfm_array_list)

# 表示するタイトルを指定します
titles = ["not processing", "processing"]
face_close_up.append(final_frames_list[0])

# 画像として表示します
show_face_frames(face_close_up, titles, figsize=(8, 5))

# ２．全体
# 後処理なし
result = get_only_swaped_fullimage(final_frames_list, crop_frames_list, full_frames[0], tfm_array_list)
noshow_images([source[0][:, :, ::-1], target_full, result], ['Source Image', 'Target Image', 'Swapped Image'], figsize=(20, 15))
